Import necessary libraries

In [4]:
from presidio_analyzer import AnalyzerEngine, PatternRecognizer, Pattern
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
import spacy.cli
from tabulate import tabulate
import spacy

Configure Mulilaguage model from configuration file

In [5]:
provider = NlpEngineProvider(conf_file="docs/languages-config.yml")
nlp_engine_multi_language = provider.create_engine()

Load or Download Models for Presidio

In [6]:
try:
    spacy.load("en_core_web_lg")
    spacy.load("de_core_news_md")
    spacy.load("es_core_news_md")
    spacy.load("it_core_news_lg")
    spacy.load("fr_core_news_sm")
except:
    spacy.cli.download("en_core_web_lg")
    spacy.cli.download("de_core_news_md")
    spacy.cli.download("es_core_news_md")
    spacy.cli.download("it_core_news_lg")
    spacy.cli.download("fr_core_news_sm")

Create list of phrases

In [7]:
phrases = [
    "Ciao, sono Maria Doe, il mio volo è ADA456. Vorrei ricevere i dettagli della mia prenotazione.",
    "Il mio nome è John Doe e il mio PNR è 212-555-5555 e la mia email è jd@test.com.",
    "Guardando nelle profondità di quell'oscurità, per molto tempo sono rimasto lì, interrogandomi, temendo, dubitando, sognando nessun sogno mortale. Hai mai osato sognare prima d'ora?"
]

Create custom PII recogniser base on sample words and context with multilanguage support

In [22]:
custom_pii_indicator_en = ['pnr']
custom_pii_context_words_en = ["Personal number reservation", "booking number", "booking code"] #context words for PII
pii_recognizer_en = PatternRecognizer(supported_entity="PNR_NUMBER", supported_language="en", deny_list=custom_pii_indicator_en, context=custom_pii_context_words_en)

custom_pii_indicator_it = ['pnr']
custom_pii_context_words_it = ["numero di prenotazione personale", "numero di prenotazione", "codice di prenotazione"] #context words for PII
pii_recognizer_it = PatternRecognizer(supported_entity="PNR_NUMBER", supported_language="it", deny_list=custom_pii_indicator_it, context=custom_pii_context_words_it)

custom_pii_indicator_es = ['pnr']
custom_pii_context_words_es = ["número de reserva personal", "número de reserva", "código de reserva"] #context words for PII
pii_recognizer_es = PatternRecognizer(supported_entity="PNR_NUMBER", supported_language="es", deny_list=custom_pii_indicator_es, context=custom_pii_context_words_es)

custom_pii_indicator_de = ['pnr']
custom_pii_context_words_de = ["persönliche Buchungsnummer", "Buchungsnummer", "Buchungscode"] #context words for PII
pii_recognizer_de = PatternRecognizer(supported_entity="PNR_NUMBER", supported_language="de", deny_list=custom_pii_indicator_de, context=custom_pii_context_words_de)

custom_pii_indicator_fr = ['pnr']
custom_pii_context_words_fr = ["numéro de réservation personnel", "numéro de réservation", "code de réservation"] #context words for PII
pii_recognizer_fr = PatternRecognizer(supported_entity="PNR_NUMBER", supported_language="fr", deny_list=custom_pii_indicator_fr, context=custom_pii_context_words_fr)

#Initialize analyzer
analyzer = AnalyzerEngine(nlp_engine=nlp_engine_multi_language, supported_languages=["en", "it", "es", "de", "fr"])
analyzer.registry.add_recognizer(pii_recognizer_en)
analyzer.registry.add_recognizer(pii_recognizer_it)
analyzer.registry.add_recognizer(pii_recognizer_es)
analyzer.registry.add_recognizer(pii_recognizer_de)
analyzer.registry.add_recognizer(pii_recognizer_fr)

#Start Analisys
for phrase in phrases:
    analyzer_results = analyzer.analyze(phrase, language="it")
    if analyzer_results:
        print(phrase)
        table = []
        for result in analyzer_results:
            value = phrase[result.start:result.end]
            table.append([result.entity_type, result.start, result.end, result.score, value])
        print(tabulate(table, headers=["entity_type", "start", "end", "score", "value"]))
        print("\n")

Ciao, sono Maria Doe, il mio volo è ADA456. Vorrei ricevere i dettagli della mia prenotazione.
entity_type      start    end    score  value
-------------  -------  -----  -------  ---------
LOCATION             0      4     0.85  Ciao
PERSON              11     20     0.85  Maria Doe


Il mio nome è John Doe e il mio PNR è 212-555-5555 e la mia email è jd@test.com.
entity_type      start    end    score  value
-------------  -------  -----  -------  ------------
PNR_NUMBER          32     35     1     PNR
EMAIL_ADDRESS       68     79     1     jd@test.com
PERSON              14     22     0.85  John Doe
PERSON              68     79     0.85  jd@test.com
URL                 71     79     0.5   test.com
PHONE_NUMBER        38     50     0.4   212-555-5555




Create custom PII recogniser base on REGEX and context with multilanguage support

In [18]:
custom_pii_regex = r"(\b[A-Za-z]{3}\d{3}\b)" #es: AZA610
custom_pii_context_words_it = ["numero di volo", "codice volo"] #context words for PII
custom_pii_context_words_en = ["flight number", "flight code"] #context words for PII
custom_pii_context_words_de = ["Flugnummer", "Flugcode"] #context words for PII
custom_pii_context_words_es = ["número de vuelo", "código de vuelo"] #context words for PII
custom_pii_context_words_fr = ["numéro de vol", "code de vol"] #context words for PII

custom_pii_pattern = Pattern(name="flight number", regex=custom_pii_regex, score=0.01)
pii_recognizer_it = PatternRecognizer(supported_entity="FLIGHT_NUMBER", patterns=[custom_pii_pattern], supported_language="it", context=custom_pii_context_words_it)
pii_recognizer_en = PatternRecognizer(supported_entity="FLIGHT_NUMBER", patterns=[custom_pii_pattern], supported_language="en", context=custom_pii_context_words_en)
pii_recognizer_de = PatternRecognizer(supported_entity="FLIGHT_NUMBER", patterns=[custom_pii_pattern], supported_language="de", context=custom_pii_context_words_de)
pii_recognizer_es = PatternRecognizer(supported_entity="FLIGHT_NUMBER", patterns=[custom_pii_pattern], supported_language="es", context=custom_pii_context_words_es)
pii_recognizer_fr = PatternRecognizer(supported_entity="FLIGHT_NUMBER", patterns=[custom_pii_pattern], supported_language="fr", context=custom_pii_context_words_fr)

#Initialize analyzer
analyzer = AnalyzerEngine(nlp_engine=nlp_engine_multi_language, supported_languages=["it", "en", "es", "de", "fr"])
analyzer.registry.add_recognizer(pii_recognizer_en)
analyzer.registry.add_recognizer(pii_recognizer_it)
analyzer.registry.add_recognizer(pii_recognizer_es)
analyzer.registry.add_recognizer(pii_recognizer_de)
analyzer.registry.add_recognizer(pii_recognizer_fr)

#Start Analisys
for phrase in phrases:
    analyzer_results = analyzer.analyze(phrase, language="it")
    if analyzer_results:
        print(phrase)
        table = []
        for result in analyzer_results:
            value = phrase[result.start:result.end]
            table.append([result.entity_type, result.start, result.end, result.score, value])
        print(tabulate(table, headers=["entity_type", "start", "end", "score", "value"]))
        print("\n")

Ciao, sono Maria Doe, il mio volo è ADA456. Vorrei ricevere i dettagli della mia prenotazione.
entity_type      start    end    score  value
-------------  -------  -----  -------  ---------
LOCATION             0      4     0.85  Ciao
PERSON              11     20     0.85  Maria Doe
FLIGHT_NUMBER       36     42     0.01  ADA456


Il mio nome è John Doe e il mio PNR è 212-555-5555 e la mia email è jd@test.com.
entity_type      start    end    score  value
-------------  -------  -----  -------  ------------
EMAIL_ADDRESS       68     79     1     jd@test.com
PERSON              14     22     0.85  John Doe
PERSON              68     79     0.85  jd@test.com
URL                 71     79     0.5   test.com
PHONE_NUMBER        38     50     0.4   212-555-5555




In [24]:
#Initialize analyzer
analyzer = AnalyzerEngine(nlp_engine=nlp_engine_multi_language, supported_languages=["it", "en", "es", "de", "fr"])

anonymizer = AnonymizerEngine()

#Start Analisys
for phrase in phrases:
    analyzer_results = analyzer.analyze(phrase, language="it")
    if analyzer_results:
        anonymized_results = anonymizer.anonymize(phrase, analyzer_results)
        print(phrase)
        print(anonymized_results.text)
        print("\n\n")

Ciao, sono Maria Doe, il mio volo è ADA456. Vorrei ricevere i dettagli della mia prenotazione.
<LOCATION>, sono <PERSON>, il mio volo è ADA456. Vorrei ricevere i dettagli della mia prenotazione.



Il mio nome è John Doe e il mio PNR è 212-555-5555 e la mia email è jd@test.com.
Il mio nome è <PERSON> e il mio PNR è <PHONE_NUMBER> e la mia email è <EMAIL_ADDRESS>.



